In [1]:
import boto3
import pandas as pd
from io import StringIO, BytesIO
from datetime import datetime, timedelta

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
arg_date = '2022-12-31'

In [4]:
arg_date_dt = datetime.strptime(arg_date, '%Y-%m-%d').date() - timedelta(days=1)

In [5]:
arg_date_dt

datetime.date(2022, 12, 30)

In [6]:
s3 = boto3.resource('s3')
bucket = s3.Bucket('xetra-1234')

In [8]:
objects = [obj for obj in bucket.objects.all() if datetime.strptime(obj.key.split('/')[0], '%Y-%m-%d').date() >= arg_date_dt ]

In [9]:
objects

[s3.ObjectSummary(bucket_name='xetra-1234', key='2022-12-30/2022-12-30_BINS_XETR00.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-12-30/2022-12-30_BINS_XETR01.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-12-30/2022-12-30_BINS_XETR02.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-12-30/2022-12-30_BINS_XETR03.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-12-30/2022-12-30_BINS_XETR04.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-12-30/2022-12-30_BINS_XETR05.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-12-30/2022-12-30_BINS_XETR06.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-12-30/2022-12-30_BINS_XETR07.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-12-30/2022-12-30_BINS_XETR08.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-12-30/2022-12-30_BINS_XETR09.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-12-30/2022-12-30_BINS_XETR10.csv'),
 s3.Object

In [10]:
csv_obj_init = bucket.Object(key=objects[0].key).get().get('Body').read().decode('utf-8')
data = StringIO(csv_obj_init)
df_init = pd.read_csv(data, delimiter=',')

In [11]:
df_init.columns

Index(['ISIN', 'Mnemonic', 'SecurityDesc', 'SecurityType', 'Currency',
       'SecurityID', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice',
       'EndPrice', 'TradedVolume', 'NumberOfTrades'],
      dtype='object')

In [12]:
df_all = pd.DataFrame(columns=df_init.columns)
for obj in objects:
    csv_obj = bucket.Object(key=obj.key).get().get('Body').read().decode('utf-8')
    data = StringIO(csv_obj)
    df = pd.read_csv(data, delimiter=',')
    df_all = pd.concat([df_all, df], ignore_index = True)


In [13]:
df_all

,ISIN,Mnemonic,SecurityDesc,SecurityType,Currency,SecurityID,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,NumberOfTrades
0,AT0000A0E9W5,SANT,S+T AG O.N.,Common stock,EUR,2504159,2022-12-30,08:00,13.880,13.880,13.880,13.880,63,1
1,DE000A0DJ6J9,S92,SMA SOLAR TECHNOL.AG,Common stock,EUR,2504287,2022-12-30,08:00,29.000,29.140,29.000,29.140,6004,29
2,DE000A0D6554,NDX1,NORDEX SE O.N.,Common stock,EUR,2504290,2022-12-30,08:00,13.760,13.880,13.760,13.800,49560,55
3,DE000A0D9PT0,MTX,MTU AERO ENGINES NA O.N.,Common stock,EUR,2504297,2022-12-30,08:00,212.000,212.000,210.400,210.600,17455,40
4,DE000A0JL9W6,VBK,VERBIO VER.BIOENERGIE ON,Common stock,EUR,2504343,2022-12-30,08:00,66.200,66.250,65.350,66.250,10884,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263833,GB00BLD4ZP54,CLTC,COINSHARES DIG.SEC.OEND,ETN,EUR,6479084,2022-12-31,16:46,19.324,19.324,19.324,19.324,0,2
263834,LU1923627332,RUSL,MUL-LYX.MSCI RUSSI.DIS.LS,ETF,EUR,5424594,2022-12-31,16:52,12.400,12.400,12.400,12.400,2645,2
263835,US98956P1021,ZIM,ZIMMER BIOMET HLDGS DL-01,Common stock,EUR,4582018,2022-12-31,20:30,113.100,113.100,113.100,113.100,0,1
263836,US9224171002,VEO,"VEECO INSTRUMENTS DL-,01",Common stock,EUR,6198311,2022-12-31,20:30,24.600,24.600,24.600,24.600,0,1


In [14]:
columns = ['ISIN', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice', 'EndPrice', 'TradedVolume']
df_all = df_all.loc[:, columns]

In [15]:
df_all.head(10)

,ISIN,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume
0,AT0000A0E9W5,2022-12-30,08:00,13.88,13.880,13.88,13.880,63
1,DE000A0DJ6J9,2022-12-30,08:00,29.00,29.140,29.00,29.140,6004
2,DE000A0D6554,2022-12-30,08:00,13.76,13.880,13.76,13.800,49560
3,DE000A0D9PT0,2022-12-30,08:00,212.00,212.000,210.40,210.600,17455
4,DE000A0JL9W6,2022-12-30,08:00,66.20,66.250,65.35,66.250,10884
5,DE000A1J5RX9,2022-12-30,08:00,2.42,2.422,2.40,2.422,75837
6,DE000A12DM80,2022-12-30,08:00,50.50,50.500,50.34,50.340,1003
7,DE000A13SX22,2022-12-30,08:00,62.08,62.440,62.02,62.020,450
8,DE000A161N30,2022-12-30,08:00,25.30,25.300,25.30,25.300,69
9,DE000KSAG888,2022-12-30,08:00,21.54,21.800,21.44,21.740,59391


In [16]:
df_all.dropna(inplace=True)
df_all.shape

(263838, 8)

## Get opening perice per ISIN and day

In [17]:
df_all['opening_price'] = df_all.sort_values(by=['Time']).groupby(['ISIN','Date'])['StartPrice'].transform('first')

In [18]:
df_all[df_all['ISIN']=='AT000000STR1']

,ISIN,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,opening_price
3135,AT000000STR1,2022-12-30,08:06,36.60,36.60,36.60,36.60,118,36.6
12433,AT000000STR1,2022-12-30,08:37,36.30,36.30,36.30,36.30,107,36.6
17595,AT000000STR1,2022-12-30,08:56,36.10,36.10,36.10,36.10,60,36.6
20877,AT000000STR1,2022-12-30,09:09,36.00,36.00,36.00,36.00,100,36.6
21425,AT000000STR1,2022-12-30,09:11,35.75,35.75,35.75,35.75,80,36.6
21959,AT000000STR1,2022-12-30,09:13,35.75,35.75,35.75,35.75,817,36.6
26216,AT000000STR1,2022-12-30,09:30,35.90,35.90,35.90,35.90,104,36.6
31570,AT000000STR1,2022-12-30,09:53,35.95,35.95,35.95,35.95,180,36.6
41273,AT000000STR1,2022-12-30,10:36,36.10,36.10,36.10,36.10,100,36.6
111942,AT000000STR1,2022-12-30,15:37,36.65,36.65,36.65,36.65,107,36.6


## Get closing perice per ISIN and day

In [19]:
df_all['closing_price'] = df_all.sort_values(by=['Time']).groupby(['ISIN','Date'])['StartPrice'].transform('last')

In [20]:
df_all[df_all['ISIN']=='AT000000STR1']

,ISIN,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,opening_price,closing_price
3135,AT000000STR1,2022-12-30,08:06,36.60,36.60,36.60,36.60,118,36.6,36.7
12433,AT000000STR1,2022-12-30,08:37,36.30,36.30,36.30,36.30,107,36.6,36.7
17595,AT000000STR1,2022-12-30,08:56,36.10,36.10,36.10,36.10,60,36.6,36.7
20877,AT000000STR1,2022-12-30,09:09,36.00,36.00,36.00,36.00,100,36.6,36.7
21425,AT000000STR1,2022-12-30,09:11,35.75,35.75,35.75,35.75,80,36.6,36.7
21959,AT000000STR1,2022-12-30,09:13,35.75,35.75,35.75,35.75,817,36.6,36.7
26216,AT000000STR1,2022-12-30,09:30,35.90,35.90,35.90,35.90,104,36.6,36.7
31570,AT000000STR1,2022-12-30,09:53,35.95,35.95,35.95,35.95,180,36.6,36.7
41273,AT000000STR1,2022-12-30,10:36,36.10,36.10,36.10,36.10,100,36.6,36.7
111942,AT000000STR1,2022-12-30,15:37,36.65,36.65,36.65,36.65,107,36.6,36.7


## Aggregations

In [21]:
df_all = df_all.groupby(['ISIN','Date'], as_index=False).agg(opening_price_eur=('opening_price', 'min'), closing_price_eur=('closing_price', 'min'), 
                                             minimum_price_eur=('MinPrice', 'min'), maximum_price_eur=('MaxPrice', 'max'),
                                            daily_raded_volume=('TradedVolume', 'sum'))

In [22]:
df_all.head(20)

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_raded_volume
0,AT000000STR1,2022-12-30,36.60,36.70,35.75,36.70,1773
1,AT000000STR1,2022-12-31,36.60,36.70,35.75,36.70,1773
2,AT00000FACC2,2022-12-30,8.05,8.57,7.87,8.57,10205
3,AT00000FACC2,2022-12-31,8.05,8.57,7.87,8.57,10205
4,AT0000606306,2022-12-30,14.51,15.00,13.65,15.28,107836
5,AT0000606306,2022-12-31,14.51,15.00,13.65,15.28,107836
6,AT0000609607,2022-12-30,11.74,12.06,11.70,12.06,1065
7,AT0000609607,2022-12-31,11.74,12.06,11.70,12.06,1065
8,AT0000644505,2022-12-30,98.20,99.20,96.10,99.20,531
9,AT0000644505,2022-12-31,98.20,99.20,96.10,99.20,531


## Percent Change Prev Closing

In [23]:
df_all['prev_closing_price'] = df_all.sort_values(by=['Date']).groupby(['ISIN'])['closing_price_eur'].shift(1)

In [24]:
df_all = df_all[df_all.Date == arg_date]

In [25]:
df_all['change_prev_closing_%'] = df_all['closing_price_eur'] - df_all['prev_closing_price'] / df_all['prev_closing_price'] * 100

In [26]:
df_all.head(10)

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_raded_volume,prev_closing_price,change_prev_closing_%
1,AT000000STR1,2022-12-31,36.60,36.70,35.75,36.70,1773,36.70,-63.30
3,AT00000FACC2,2022-12-31,8.05,8.57,7.87,8.57,10205,8.57,-91.43
5,AT0000606306,2022-12-31,14.51,15.00,13.65,15.28,107836,15.00,-85.00
7,AT0000609607,2022-12-31,11.74,12.06,11.70,12.06,1065,12.06,-87.94
9,AT0000644505,2022-12-31,98.20,99.20,96.10,99.20,531,99.20,-0.80
11,AT0000652011,2022-12-31,32.17,32.40,30.66,32.70,35792,32.40,-67.60
13,AT0000652250,2022-12-31,22.70,22.95,22.70,22.95,331,22.95,-77.05
15,AT0000730007,2022-12-31,40.32,39.00,38.76,40.32,3292,39.00,-61.00
17,AT0000743059,2022-12-31,44.36,43.10,40.85,44.36,31274,43.10,-56.90
19,AT0000746409,2022-12-31,104.00,108.40,102.50,108.40,538,108.40,8.40


In [27]:
df_all.drop(columns=['prev_closing_price'], inplace=True)

In [28]:
df_all = df_all.round(decimals=2)

## Write to S3

In [29]:
key = 'xetra_daily_report_' + datetime.today().strftime('%Y%m%d_%H%M%S') + '.parquet'

In [30]:
out_buffer = BytesIO()
df_all.to_parquet(out_buffer, index=False)

In [32]:
bucket_target = s3.Bucket('udemy-xetra-1234')

In [35]:
bucket_target.put_object(Body=out_buffer.getvalue(), Key=key)

s3.Object(bucket_name='udemy-xetra-1234', key='xetra_daily_report_20241127_232202.parquet')

## Reading the uploaded file

In [37]:
for obj in bucket_target.objects.all():
    print(obj.key)

xetra_daily_report_20241127_232202.parquet


In [39]:
prq_obj = bucket_target.Object(key='xetra_daily_report_20241127_232202.parquet').get().get('Body').read()
data = BytesIO(prq_obj)
df_report = pd.read_parquet(data)

In [40]:
df_report.head(5)

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_raded_volume,change_prev_closing_%
0,AT000000STR1,2022-12-31,36.60,36.70,35.75,36.70,1773,-63.30
1,AT00000FACC2,2022-12-31,8.05,8.57,7.87,8.57,10205,-91.43
2,AT0000606306,2022-12-31,14.51,15.00,13.65,15.28,107836,-85.00
3,AT0000609607,2022-12-31,11.74,12.06,11.70,12.06,1065,-87.94
4,AT0000644505,2022-12-31,98.20,99.20,96.10,99.20,531,-0.80
